In [14]:
import pandas as pd
import re
from io import StringIO

In [26]:
# Import Data
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

df = pd.read_csv(StringIO(data), delimiter=';')
print(df)

         Airline Code    DelayTimes  FlightCodes           To_From
0      Air Canada (!)      [21, 40]      20015.0  WAterLoo_NEWYork
1   <Air France> (12)            []          NaN  Montreal_TORONTO
2  (Porter Airways. )  [60, 22, 87]      20035.0    CALgary_Ottawa
3      12. Air France      [78, 66]          NaN  Ottawa_VANcouvER
4   ".\.Lufthansa.\."      [12, 33]      20055.0   london_MONTreal


In [27]:
# Functions
def split_to_from(value):
    if pd.isna(value):
        return pd.Series([None, None])
    
    parts = value.split('_')
    from_city = parts[0].strip().title()
    to_city = parts[1].strip().title()
    
    return pd.Series([from_city, to_city])


def airline_code(code):
    code = str(code)
    code = re.sub(r'[^a-zA-Z ]+', '', code).strip()
    return code


In [28]:
# 1. FlightCodes column
df['FlightCodes'] = pd.to_numeric(df['FlightCodes'], errors='coerce')
min_val = int(df['FlightCodes'].min())
flight_codes = [min_val + i * 10 for i in range(len(df))]
df['FlightCodes'] = flight_codes

print(df)


         Airline Code    DelayTimes  FlightCodes           To_From
0      Air Canada (!)      [21, 40]        20015  WAterLoo_NEWYork
1   <Air France> (12)            []        20025  Montreal_TORONTO
2  (Porter Airways. )  [60, 22, 87]        20035    CALgary_Ottawa
3      12. Air France      [78, 66]        20045  Ottawa_VANcouvER
4   ".\.Lufthansa.\."      [12, 33]        20055   london_MONTreal


In [29]:
#2. To_From column
df[['From', 'To']] = df['To_From'].apply(split_to_from)
df.drop(columns=['To_From'], inplace=True)

print(df)


         Airline Code    DelayTimes  FlightCodes      From         To
0      Air Canada (!)      [21, 40]        20015  Waterloo    Newyork
1   <Air France> (12)            []        20025  Montreal    Toronto
2  (Porter Airways. )  [60, 22, 87]        20035   Calgary     Ottawa
3      12. Air France      [78, 66]        20045    Ottawa  Vancouver
4   ".\.Lufthansa.\."      [12, 33]        20055    London   Montreal


In [30]:
#3. Airline Code column
df['Airline Code'] = df['Airline Code'].apply(airline_code)

print(df)

     Airline Code    DelayTimes  FlightCodes      From         To
0      Air Canada      [21, 40]        20015  Waterloo    Newyork
1      Air France            []        20025  Montreal    Toronto
2  Porter Airways  [60, 22, 87]        20035   Calgary     Ottawa
3      Air France      [78, 66]        20045    Ottawa  Vancouver
4       Lufthansa      [12, 33]        20055    London   Montreal
